In [10]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Define image dimensions
image_dimensions = {'height': 256, 'width': 256, 'channels': 3}

# Create a Classifier class
class Classifier:
    def __init__(self):
        self.model = None
    
    def predict(self, x):
        return self.model.predict(x)
    
    def fit(self, x, y):
        return self.model.train_on_batch(x, y)
    
    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)
    
    def load(self, path):
        self.model.load_weights(path)

# Create a MesoNet class using the Classifier
class Meso4(Classifier):
    def __init__(self, learning_rate=0.001):
        self.model = self.init_model()
        optimizer = Adam(lr=learning_rate)
        self.model.compile(optimizer=optimizer,
                           loss='binary_crossentropy',
                           metrics=['accuracy'])
    
    def init_model(self): 
        x = Input(shape=(image_dimensions['height'],
                         image_dimensions['width'],
                         image_dimensions['channels']))
        
        x1 = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
        x2 = Conv2D(8, (5, 5), padding='same', activation='relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)
        
        x3 = Conv2D(16, (5, 5), padding='same', activation='relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
        x4 = Conv2D(16, (5, 5), padding='same', activation='relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation='sigmoid')(y)
        
        return Model(inputs=x, outputs=y)

# Instantiate a MesoNet model with pretrained weights
meso = Meso4()
meso.load('./weights/Meso4_DF')
# meso.load('./Mesonet/weights/Meso4_F2F.h5')


In [11]:
'''
# Prepare image data
dataGenerator = ImageDataGenerator(rescale=1./255)

# Creating training, validation, and test data generators
train_generator = dataGenerator.flow_from_directory(
    './data/train/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

validation_generator = dataGenerator.flow_from_directory(
    './data/validation/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

test_generator = dataGenerator.flow_from_directory(
    './data/test/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)
'''

"\n# Prepare image data\ndataGenerator = ImageDataGenerator(rescale=1./255)\n\n# Creating training, validation, and test data generators\ntrain_generator = dataGenerator.flow_from_directory(\n    './data/train/',\n    target_size=(256, 256),\n    batch_size=32,\n    class_mode='binary'\n)\n\nvalidation_generator = dataGenerator.flow_from_directory(\n    './data/validation/',\n    target_size=(256, 256),\n    batch_size=32,\n    class_mode='binary'\n)\n\ntest_generator = dataGenerator.flow_from_directory(\n    './data/test/',\n    target_size=(256, 256),\n    batch_size=32,\n    class_mode='binary'\n)\n"

In [12]:
'''
# Training the model
num_epochs = 10
steps_per_epoch = len(train_generator)

for epoch in range(num_epochs):
    for step in range(steps_per_epoch):
        x_batch, y_batch = train_generator.next()  # Get next batch of data
        loss, accuracy = meso.fit(x_batch, y_batch)  # Train on this batch
        if step % 10 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, Step {step + 1}/{steps_per_epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
    
    # Evaluate on validation set after each epoch
    val_loss, val_accuracy = meso.get_accuracy(validation_generator.next()[0], validation_generator.next()[1])
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

# Final evaluation on the test set
test_loss, test_accuracy = meso.get_accuracy(test_generator.next()[0], test_generator.next()[1])
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
'''

'\n# Training the model\nnum_epochs = 10\nsteps_per_epoch = len(train_generator)\n\nfor epoch in range(num_epochs):\n    for step in range(steps_per_epoch):\n        x_batch, y_batch = train_generator.next()  # Get next batch of data\n        loss, accuracy = meso.fit(x_batch, y_batch)  # Train on this batch\n        if step % 10 == 0:\n            print(f"Epoch {epoch + 1}/{num_epochs}, Step {step + 1}/{steps_per_epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")\n    \n    # Evaluate on validation set after each epoch\n    val_loss, val_accuracy = meso.get_accuracy(validation_generator.next()[0], validation_generator.next()[1])\n    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")\n\n# Final evaluation on the test set\ntest_loss, test_accuracy = meso.get_accuracy(test_generator.next()[0], test_generator.next()[1])\nprint(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")\n'